## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

In [1]:
import pandas as pd
import numpy as np

import pyspark

In [2]:
pyspark.__version__

'3.0.2'

# Подготовка данных

# Обучение моделей

# Анализ результатов